### Step 1: Install Ultralytics Library
This cell installs the Ultralytics library, which is required for training and inference with YOLO models. The `-q` flag ensures the installation is quiet, avoiding unnecessary output.

In [ ]:
!pip install ultralytics -q

### Step 2: Train the Segmentation Model
This cell loads a pre-trained YOLOv8 Nano segmentation model (`yolov8n-seg.pt`) and fine-tunes it on a custom dataset defined in `carparts-seg.yaml`.

#### Improvements:
1. **Added `project` and `name`**: Organizes training outputs into a specific folder.
2. **Added `batch`**: Specifies the batch size for training.
3. **Added `workers`**: Speeds up data loading by using multiple workers.
4. **Added `device`**: Ensures training uses GPU if available.

In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLO segmentation model
model = YOLO("yolov8n-seg.pt")  # Use a pre-trained YOLOv8 Nano segmentation model

# Train the model
results = model.train(
    data="carparts-seg.yaml",  # Path to your dataset configuration file
    epochs=60,                # Number of training epochs
    imgsz=640,                # Image size for training
    batch=16,                 # Batch size for training
    workers=4,                # Number of data loading workers
    device="0",               # Use GPU (if available)
    project="carparts-seg",   # Save results in a specific project folder
)

### Step 3: Validate the Model
This cell validates the trained model on the validation set to evaluate its performance. Metrics such as mAP (mean Average Precision), precision, and recall are printed.

#### Improvements:
1. **Added `split`**: Explicitly specifies the validation split.
2. **Added `conf`**: Sets a confidence threshold for validation.

In [ ]:
# Validate the model
metrics = model.val(
    split="val",  # Validate on the validation set
    conf=0.5,    # Confidence threshold for validation
)
print(metrics)   # Print evaluation metrics (mAP, precision, recall, etc.)

### Step 4: Mount Google Drive
This cell mounts Google Drive to access external images stored in Predict Images for prediction.

In [ ]:
# Mount Google Drive to access the dataset for the predicted images (outside images)
from google.colab import drive
drive.mount('/content/drive')

### Step 5: Predict on External Images
This cell loads the best-trained model (`best.pt`) and uses it to predict on external images stored in Google Drive. The images are resized to 640x640 pixels before prediction.

#### Improvements:
1. **Added `project` and `name`**: Organizes prediction outputs into a specific folder.
2. **Added `batch`**: Processes multiple images at once for faster inference.
3. **Added `device`**: Ensures prediction uses GPU if available.
4. **Added `exist_ok`**: Prevents errors if the output folder already exists.

In [ ]:
# Load the best trained model
best_model = YOLO("carparts-seg/train/weights/best.pt")  # Path to your trained model

# Path to the folder containing images for prediction
predict_images_path = "/content/drive/MyDrive/Predict Images/"

# Predict on images with resizing
results = best_model.predict(
    source=predict_images_path,  # Path to the folder with images
    imgsz=640,                  # Resize images to 640x640 before prediction
    save=True,                  # Save the prediction results
    save_txt=True,             # Save the prediction labels (optional)
    conf=0.5,                  # Confidence threshold (optional)
    batch=8,                   # Process 8 images at a time
    device="0",                # Use GPU (if available)
    project="carparts-seg",    # Save results in a specific project folder
    name="predict-v1",         # Name of the prediction run
    exist_ok=True,             # Overwrite existing files if needed
)

# Display results
for result in results:
    result.show()  # Show the prediction results